- we need to check & temperature
- we can get pricing data (Noel)
- we should try to establish the accuracy of current forecast demand vs actual
- temperature data...
- what does AEMO say? residential & commercial demand, temperature, 
- we should start by replicating the AEMO
- spectral analysis reading from william
- 

In [1]:
import pandas as pd
from pandas.plotting import autocorrelation_plot
import os
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
# general settings
class CFG:
    data_path = './../data'
    img_dim1 = 20
    img_dim2 = 10
    

# adjust the parameters for displayed figures
plt.style.use('fivethirtyeight')
plt.rcParams.update({'figure.figsize': (CFG.img_dim1, CFG.img_dim2)})  

## NSW Forecast Demand

In [3]:
nsw_forecastdemand = pd.read_csv(
    os.path.join(CFG.data_path, 'NSW', 'forecastdemand_nsw.csv'), 
    parse_dates=['LASTCHANGED', 'DATETIME']
)

In [4]:
nsw_forecastdemand.head(10)

In [5]:
nsw_forecastdemand.tail()

In [6]:
nsw_forecastdemand.info()

In [7]:
nsw_forecastdemand['FORECASTDEMAND'].describe()

In [8]:
nsw_forecastdemand.shape

In [9]:
nsw_forecastdemand.min()

In [10]:
nsw_forecastdemand.max()

In [11]:
nsw_forecastdemand.isnull().sum()

In [12]:
nsw_forecastdemand.isna().sum()

In [13]:
nsw_forecastdemand.nunique()

In [14]:
nsw_forecastdemand.dtypes

## NSW temperature

In [15]:
nsw_temperature = pd.read_csv(
    os.path.join(CFG.data_path, 'NSW', 'temperature_nsw.csv'), 
    parse_dates=['DATETIME'], 
    dayfirst=True
)

In [16]:
nsw_temperature['DATE'] = pd.to_datetime(nsw_temperature['DATETIME'])

In [17]:
nsw_temperature.dtypes

In [18]:
nsw_temperature[['DATE', 'TEMPERATURE']].set_index('DATE').resample('30min').mean()

In [19]:
# nsw_temperature.groupby(pd.Grouper(key='DATE', freq='30min')).mean().reset_index()

In [20]:
nsw_temperature.head(10)

In [21]:
nsw_temperature.tail(10)

In [22]:
nsw_temperature.info()

In [23]:
nsw_temperature['TEMPERATURE'].describe()

In [24]:
nsw_temperature.shape

In [25]:
nsw_temperature.min()

In [26]:
nsw_temperature.max()

In [27]:
nsw_temperature.isnull().sum()

In [28]:
nsw_temperature.isna().sum()

In [29]:
nsw_temperature.nunique()

In [30]:
nsw_temperature.dtypes

## NSW Total Demand

In [31]:
nsw_totaldemand = pd.read_csv(
    os.path.join(CFG.data_path, 'NSW', 'totaldemand_nsw.csv'), 
    parse_dates=['DATETIME'], 
    dayfirst=True
)

In [32]:
nsw_totaldemand.head()

In [33]:
nsw_totaldemand.tail()

In [34]:
nsw_totaldemand.info()

In [35]:
nsw_totaldemand['TOTALDEMAND'].describe()

In [36]:
nsw_totaldemand.shape

In [37]:
nsw_totaldemand.min()

In [38]:
nsw_totaldemand.max()

In [39]:
nsw_totaldemand.isnull().sum()

In [40]:
nsw_totaldemand.isna().sum()

In [41]:
nsw_totaldemand.nunique()

In [42]:
nsw_totaldemand.dtypes

## merge step 1

In [43]:
merged_df = pd.merge(
    nsw_totaldemand[['DATETIME', 'TOTALDEMAND']], 
    nsw_forecastdemand[['FORECASTDEMAND', 'DATETIME']], 
    on=['DATETIME'],
    how='inner'
)

# fix_me: need to group by 30min and then merge dataframes together. left inner join is not great. drop location column

In [44]:
merged_df.head(100)

In [45]:
merged_df.isnull().sum()

## merge step 2

In [46]:
nsw_df = pd.merge(
    merged_df, 
    nsw_temperature[['DATE', 'TEMPERATURE']],
    on='DATE', 
    how='outer'
)

# fix_me: the sampling rate instead of using inner join, which drops about 26k rows.
#  bucketing
#  groupby, fill using mean? or dropna to keep to the 30 min interval? 
#  we need to get special days
#  we need to get the temperature data
#  we need to get the pricing data

## Quick Check of merged df

In [ ]:
nsw_df.isnull().sum()

In [ ]:
nsw_df.head(5)

In [ ]:
nsw_df.tail(5)

In [ ]:
nsw_df.shape

In [ ]:
nsw_df.info()

In [ ]:
# nsw_df.min()

In [ ]:
# nsw_df.max()

In [ ]:
nsw_df.describe()

In [ ]:
nsw_df.nunique()

In [ ]:
nsw_df.dtypes

In [ ]:
nsw_df.isna().sum()

In [ ]:
nsw_df.to_parquet(
    os.path.join(CFG.data_path, 'NSW', 'nsw_df.parquet'),
    engine='pyarrow', 
    compression='snappy'
)

In [ ]:
nsw_df.to_csv(
    os.path.join(CFG.data_path, 'NSW', 'nsw_df.csv'), 
    header=True
)

## 

## Plots

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(nsw_df['DATETIME'], nsw_df['FORECASTDEMAND'], label='Forecast Demand')
plt.plot(nsw_df['DATETIME'], nsw_df['TOTALDEMAND'], label='Actual Demand', alpha=0.7)
plt.xlabel('Datetime')
plt.ylabel('Price')
plt.title('Forecast vs Actual Price Over Time')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(nsw_df['DATETIME'], nsw_df['TEMPERATURE'], color='orange', label='TEMPERATURE')
plt.xlabel('Datetime')
plt.ylabel('Temperature')
plt.title('Temperature Over Time')
plt.legend()
plt.show()

## scatter plot

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(nsw_df['TEMPERATURE'], nsw_df['TOTALDEMAND'], label='Actual Demand', alpha=0.5)
plt.scatter(nsw_df['TEMPERATURE'], nsw_df['FORECASTDEMAND'], label='Forecast Demand', alpha=0.5)
plt.xlabel('Temperature')
plt.ylabel('Demand')
plt.title('Temperature vs Demand')
plt.legend()
plt.show()

## Box plot

In [ ]:
plt.figure(figsize=(12, 6))
nsw_df[['FORECASTDEMAND', 'TOTALDEMAND', 'TEMPERATURE']].plot(kind='box')
plt.title('Distribution of Prices and Temperature')
plt.show()

## Histogram

In [ ]:
plt.figure(figsize=(14, 7))
nsw_df['TOTALDEMAND'].hist(alpha=0.7, label='Actual Price', bins=20)
nsw_df['FORECASTDEMAND'].hist(alpha=0.5, label='Forecast Price', bins=20)
plt.xlabel('Demand')
plt.ylabel('Frequency')
plt.title('Frequency Distribution of Demand')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
nsw_df['TEMPERATURE'].hist(color='orange', bins=20)
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.title('Frequency Distribution of Temperature')
plt.show()

## Correlation 

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(nsw_df[['FORECASTDEMAND', 'TOTALDEMAND', 'TEMPERATURE']].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

## seasonal decomposition (additive)
It's important to have a defined frequency (e.g., 'H' for hourly, 'D' for daily). Let's assume it's hourly data for this example

In [ ]:
nsw_df = nsw_df.asfreq('30min')

Now, choose the demand for electricity for decomposition

In [ ]:
ts = nsw_df['TOTALDEMAND']  # fix_me: missing values
ts.head()

In [ ]:
ts.isnull().sum()

In [ ]:
ts.isna().sum()

Perform seasonal decomposition. 
compare `model`=`multiplicative` vs `model`=`additive` if necessary

In [ ]:
decomposition = sm.tsa.seasonal_decompose(ts, model='additive')   
# fix_me: deal with missing values, temporarily using ffill

Plot the decomposed components of the time series

In [ ]:
fig = decomposition.plot()
plt.show()

## additive vs multiplicative relationships between components